#### Cosine Similarity (Qwen3-Embedding version 1)

This time I am going to use another model to compare to the other versions of the "Recommendations engine". The model will be Qwen3-Embedding-6B. A model developed by Alibaba for embeddings and tops most charts for embeddings. It is much heavier and has more parameters and longer context window. It is expected to perform the best compared to other models.

In [1]:
import pandas as pd

In [2]:
sg_df_clean = pd.read_csv("sg_df_clean.csv")

In [4]:
from sentence_transformers import SentenceTransformer
import torch
import gc

# Load the model 
model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B", 
    device="cuda", 
    model_kwargs={"torch_dtype": torch.bfloat16} #using bf16 instead of f32 or f64 for efficiency
)

# Set the max sequence length (Qwen3 supports up to 32k, but 512 is plenty)
model.max_seq_length = 512 

# Format the text with the instruction
print("Preparing text for Qwen3...")
instruction = "Instruct: Given a game description, find similar games\nQuery: "
texts = [instruction + str(text) for text in sg_df_clean['about_the_game']]

# Generate Embeddings
print("Starting Qwen3 Encoding in BF16 (Turbo Mode)...")
embeddings = model.encode(
    texts, 
    batch_size=64, #64 to make sure that it runs on 16GB VRAM
    show_progress_bar=True, 
    normalize_embeddings=True
)

# Save to DataFrame
sg_df_clean['embeddings_qwen'] = list(embeddings)

# Immediate VRAM Cleanup
del model
gc.collect()
torch.cuda.empty_cache()

print("\nEncoding complete. GPU VRAM has been cleared.")

`torch_dtype` is deprecated! Use `dtype` instead!


Preparing text for Qwen3...
Starting Qwen3 Encoding in BF16 (Turbo Mode)...


Batches:   0%|          | 0/1131 [00:00<?, ?it/s]


Encoding complete. GPU VRAM has been cleared.


In [7]:
sg_df_clean['embeddings_qwen']

0        [0.03112793, -0.067871094, -0.006072998, -0.02...
1        [0.059814453, -0.061279297, -0.00028419495, -0...
2        [0.021240234, -0.05444336, -0.0024414062, -0.0...
3        [0.07373047, -0.12597656, -0.0035247803, -0.01...
4        [0.049560547, -0.047851562, -0.004638672, -0.0...
                               ...                        
72366    [0.0043640137, -0.044921875, -0.008483887, -0....
72367    [0.06933594, -0.05078125, -0.0010757446, -0.07...
72368    [0.028320312, -0.018432617, -0.0020446777, -0....
72369    [0.09082031, -0.04711914, -0.0022277832, -0.05...
72370    [-0.0066223145, -0.076660156, -0.0028839111, -...
Name: embeddings_qwen, Length: 72371, dtype: object

In [9]:
import numpy as np

In [14]:
embedding_matrix = np.array(sg_df_clean["embeddings_qwen"].to_list()) #we need to "unpack" the column to create a matrix to use for cosine similarity

__Cosine Similarity__

We are now repeating similar steps to previous documents

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
sim_matrix_embeddings = cosine_similarity(embedding_matrix) #cosine similarity for the new embeddings

In [19]:
tfidf_matrix = pd.read_pickle("TF-IDF_V1.pkl")

In [20]:
sim_matrix_tfidf = cosine_similarity(tfidf_matrix) #cosine similarity for tf-idf

In [24]:
del tfidf_matrix
del embedding_matrix

In [26]:
import gc
gc.collect() 

0

In [28]:
del sg_df_clean

In [30]:
sim_matrix_embeddings = sim_matrix_embeddings.astype(np.float32)

In [31]:
sim_matrix_tfidf = sim_matrix_tfidf.astype(np.float32)

In [33]:
tfidf_w = 0.3
embeddings_w = 0.7

In [34]:
weighted_matrix_1 = tfidf_w * sim_matrix_tfidf

In [35]:
del sim_matrix_tfidf
gc.collect()

0

In [36]:
weighted_matrix_2 =embeddings_w * sim_matrix_embeddings

In [37]:
del sim_matrix_embeddings

In [38]:
gc.collect()

0

In [39]:
final_matrix = weighted_matrix_1 + weighted_matrix_2

In [40]:
del weighted_matrix_1 
del weighted_matrix_2

In [41]:
pd.to_pickle(final_matrix, "Full_cosine_matrix_Qwen-3-6B.pkl")